# 🔍 CodeSense AI - Development Notebook

**AI-Powered Code Review Assistant**

This notebook allows you to develop and test CodeSense AI on Kaggle.

Author: Amr Hassan | [GitHub](https://github.com/amrgaberM) | [LinkedIn](https://linkedin.com/in/amrhassangaber)

## 1. Setup & Installation

In [ ]:
# Install required packages
!pip install -q groq langchain langchain-groq pydantic pydantic-settings rich

In [ ]:
# Set your API key (get free key at https://console.groq.com/keys)
import os
from kaggle_secrets import UserSecretsClient

# Option 1: From Kaggle Secrets (recommended)
try:
    user_secrets = UserSecretsClient()
    os.environ['GROQ_API_KEY'] = user_secrets.get_secret('GROQ_API_KEY')
    print('✅ API key loaded from Kaggle Secrets')
except:
    # Option 2: Set manually (don't commit this!)
    os.environ['GROQ_API_KEY'] = 'your_api_key_here'
    print('⚠️ Set your API key manually')

## 2. Core Components

In [ ]:
# Data Models
from enum import Enum
from typing import Optional
from pydantic import BaseModel, Field
from datetime import datetime


class Severity(str, Enum):
    CRITICAL = "critical"
    HIGH = "high"
    MEDIUM = "medium"
    LOW = "low"
    INFO = "info"


class IssueCategory(str, Enum):
    SECURITY = "security"
    BUG = "bug"
    PERFORMANCE = "performance"
    STYLE = "style"
    BEST_PRACTICE = "best_practice"
    DOCUMENTATION = "documentation"


class Issue(BaseModel):
    title: str
    description: str
    severity: Severity
    category: IssueCategory
    line_start: Optional[int] = None
    line_end: Optional[int] = None
    code_snippet: Optional[str] = None
    suggestion: Optional[str] = None
    suggested_code: Optional[str] = None


class FileReview(BaseModel):
    filename: str
    language: str
    lines_of_code: int
    issues: list[Issue] = []
    summary: Optional[str] = None

print('✅ Models defined')

In [ ]:
# Prompts
SYSTEM_PROMPT = """You are CodeSense AI, an expert code reviewer with deep knowledge of software engineering best practices, security vulnerabilities, and code quality standards.

Your role is to analyze code and provide detailed, actionable feedback. You should:

1. **Security Analysis**: Identify potential security vulnerabilities (OWASP Top 10, injection attacks, etc.)
2. **Bug Detection**: Find logic errors, off-by-one errors, null pointer issues, race conditions
3. **Performance**: Identify inefficient algorithms, unnecessary computations, memory leaks
4. **Code Quality**: Check for code smells, duplications, complexity issues
5. **Best Practices**: Ensure adherence to language-specific conventions

Be thorough but practical. Focus on issues that matter most."""

CODE_REVIEW_PROMPT = """Analyze the following {language} code and identify issues.

**Filename:** {filename}

```{language}
{code}
```

Provide your analysis as a JSON object with this exact structure:
{{
    "summary": "Brief overall assessment (2-3 sentences)",
    "quality_score": <number 0-100>,
    "issues": [
        {{
            "title": "Short descriptive title",
            "description": "Detailed explanation",
            "severity": "critical|high|medium|low|info",
            "category": "security|bug|performance|style|best_practice|documentation",
            "line_start": <line number or null>,
            "line_end": <line number or null>,
            "code_snippet": "relevant code or null",
            "suggestion": "How to fix",
            "suggested_code": "Fixed code example or null"
        }}
    ]
}}

Respond ONLY with the JSON object, no additional text."""

print('✅ Prompts defined')

In [ ]:
# LLM Client
import json
import re
from groq import Groq


class CodeReviewLLM:
    def __init__(self, model: str = "llama-3.3-70b-versatile"):
        self.client = Groq(api_key=os.environ.get('GROQ_API_KEY'))
        self.model = model
    
    def _parse_response(self, content: str) -> dict:
        """Extract JSON from LLM response."""
        # Remove markdown code blocks if present
        json_match = re.search(r'```(?:json)?\s*([\s\S]*?)\s*```', content)
        if json_match:
            content = json_match.group(1)
        
        content = content.strip()
        
        try:
            return json.loads(content)
        except json.JSONDecodeError:
            # Fix common JSON issues
            content = re.sub(r',\s*}', '}', content)
            content = re.sub(r',\s*]', ']', content)
            return json.loads(content)
    
    def review(self, code: str, language: str = "python", filename: str = "code") -> dict:
        """Review code and return analysis."""
        user_prompt = CODE_REVIEW_PROMPT.format(
            code=code,
            language=language,
            filename=filename
        )
        
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.1,
            max_tokens=4096,
        )
        
        content = response.choices[0].message.content
        return self._parse_response(content)


# Initialize
llm = CodeReviewLLM()
print('✅ LLM client initialized')

In [ ]:
# Code Analyzer
from typing import Any


class CodeAnalyzer:
    def __init__(self):
        self.llm = CodeReviewLLM()
    
    def _parse_issue(self, issue_data: dict) -> Issue:
        """Convert dict to Issue model."""
        severity_str = issue_data.get("severity", "medium").lower()
        try:
            severity = Severity(severity_str)
        except ValueError:
            severity = Severity.MEDIUM
        
        category_str = issue_data.get("category", "best_practice").lower()
        try:
            category = IssueCategory(category_str)
        except ValueError:
            category = IssueCategory.BEST_PRACTICE
        
        return Issue(
            title=issue_data.get("title", "Unknown Issue"),
            description=issue_data.get("description", "No description"),
            severity=severity,
            category=category,
            line_start=issue_data.get("line_start"),
            line_end=issue_data.get("line_end"),
            code_snippet=issue_data.get("code_snippet"),
            suggestion=issue_data.get("suggestion"),
            suggested_code=issue_data.get("suggested_code")
        )
    
    def review(self, code: str, language: str = "python", filename: str = "code") -> FileReview:
        """Review code and return structured results."""
        lines_of_code = len(code.splitlines())
        
        # Get LLM analysis
        result = self.llm.review(code, language, filename)
        
        # Parse issues
        issues = []
        for issue_data in result.get("issues", []):
            try:
                issues.append(self._parse_issue(issue_data))
            except Exception:
                continue
        
        return FileReview(
            filename=filename,
            language=language,
            lines_of_code=lines_of_code,
            issues=issues,
            summary=result.get("summary")
        )


# Initialize
analyzer = CodeAnalyzer()
print('✅ Analyzer ready')

In [ ]:
# Pretty Printer
from rich.console import Console
from rich.panel import Panel
from rich.table import Table
from rich.syntax import Syntax

console = Console()

SEVERITY_STYLES = {
    Severity.CRITICAL: ("red bold", "🔴"),
    Severity.HIGH: ("orange1", "🟠"),
    Severity.MEDIUM: ("yellow", "🟡"),
    Severity.LOW: ("blue", "🔵"),
    Severity.INFO: ("dim", "⚪"),
}


def print_review(review: FileReview):
    """Print review with rich formatting."""
    # Header
    console.print(Panel(
        f"[bold]{review.filename}[/bold]\n"
        f"Language: {review.language} | Lines: {review.lines_of_code}",
        title="📄 Code Review",
        border_style="cyan"
    ))
    
    # Summary
    if review.summary:
        console.print(f"\n[italic]{review.summary}[/italic]\n")
    
    # No issues
    if not review.issues:
        console.print("[green]✅ No issues found! Looking good.[/green]\n")
        return
    
    # Issues
    for i, issue in enumerate(review.issues, 1):
        style, emoji = SEVERITY_STYLES[issue.severity]
        
        console.print(f"{emoji} [bold {style}]Issue #{i}: {issue.title}[/bold {style}]")
        console.print(f"   [dim]Severity:[/dim] [{style}]{issue.severity.value.upper()}[/{style}] | "
                     f"[dim]Category:[/dim] {issue.category.value}")
        
        if issue.line_start:
            line_info = f"Line {issue.line_start}"
            if issue.line_end and issue.line_end != issue.line_start:
                line_info += f"-{issue.line_end}"
            console.print(f"   [dim]Location:[/dim] {line_info}")
        
        console.print(f"\n   {issue.description}")
        
        if issue.code_snippet:
            console.print("\n   [dim]Problematic code:[/dim]")
            syntax = Syntax(issue.code_snippet, review.language, theme="monokai", padding=1)
            console.print(syntax)
        
        if issue.suggestion:
            console.print(f"\n   [green]💡 Suggestion:[/green] {issue.suggestion}")
        
        if issue.suggested_code:
            console.print("\n   [green]Fixed code:[/green]")
            syntax = Syntax(issue.suggested_code, review.language, theme="monokai", padding=1)
            console.print(syntax)
        
        console.print("\n" + "─" * 60 + "\n")

print('✅ Pretty printer ready')

## 3. Test the Code Reviewer!

In [ ]:
# Example 1: Simple Python code with issues
test_code_1 = """
import pickle
import os

def process_user_data(user_input):
    # Execute user command
    os.system(user_input)
    
    # Load data from untrusted source
    with open('data.pkl', 'rb') as f:
        data = pickle.load(f)
    
    return data

def divide(a, b):
    return a / b

password = "admin123"
"""

print("🔍 Reviewing code...\n")
result = analyzer.review(test_code_1, "python", "vulnerable_code.py")
print_review(result)

In [ ]:
# Example 2: JavaScript code
test_code_2 = """
const express = require('express');
const app = express();

app.get('/user', (req, res) => {
    const userId = req.query.id;
    const query = `SELECT * FROM users WHERE id = ${userId}`;
    
    db.query(query, (err, result) => {
        res.send(result);
    });
});

app.post('/login', (req, res) => {
    const password = req.body.password;
    if (password == "admin") {
        res.send("Logged in!");
    }
});
"""

print("🔍 Reviewing JavaScript code...\n")
result = analyzer.review(test_code_2, "javascript", "server.js")
print_review(result)

In [ ]:
# Example 3: Good code (should have few or no issues)
test_code_3 = """
from typing import Optional
import logging

logger = logging.getLogger(__name__)


def divide(numerator: float, denominator: float) -> Optional[float]:
    \"\"\"Safely divide two numbers.
    
    Args:
        numerator: The number to divide
        denominator: The number to divide by
    
    Returns:
        The result of division, or None if denominator is zero
    \"\"\"
    if denominator == 0:
        logger.warning("Attempted division by zero")
        return None
    return numerator / denominator
"""

print("🔍 Reviewing good code...\n")
result = analyzer.review(test_code_3, "python", "safe_math.py")
print_review(result)

## 4. Review Your Own Code

In [ ]:
# Paste your code here!
your_code = """
# Paste your code here
def hello():
    print("Hello, World!")
"""

# Set the language
language = "python"  # python, javascript, java, go, etc.
filename = "my_code.py"

# Review!
print("🔍 Reviewing your code...\n")
result = analyzer.review(your_code, language, filename)
print_review(result)

## 5. Export Results

In [ ]:
# Export last review to JSON
import json

export_data = {
    "filename": result.filename,
    "language": result.language,
    "lines_of_code": result.lines_of_code,
    "summary": result.summary,
    "total_issues": len(result.issues),
    "issues": [issue.model_dump() for issue in result.issues]
}

with open('review_result.json', 'w') as f:
    json.dump(export_data, f, indent=2, default=str)

print('✅ Results saved to review_result.json')

---

## 🚀 Next Steps

1. **Clone the full repo** for CLI and API access
2. **Add GitHub integration** for automatic PR reviews
3. **Deploy to Railway/Render** for production use

GitHub: [github.com/amrgaberM/codesense-ai](https://github.com/amrgaberM/codesense-ai)